In [12]:
import os
import re
from functools import reduce
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, regexp_replace, year, avg, to_timestamp

In [2]:
spark = SparkSession.builder.appName("Bronze_to_Silver").getOrCreate()
spark.sql("CREATE NAMESPACE IF NOT EXISTS datalake.silver")

25/03/25 16:56:51 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


DataFrame[]

In [4]:
df_gold_silver = spark.read.format("iceberg").table("datalake.silver.gold")
df_gold_silver.show(10)

+---+----+-----+-----+-----+-----+
| ID|Date|Price| Open| High|  Low|
+---+----+-----+-----+-----+-----+
|  1|NULL|332.5|333.0|333.4|332.3|
|  2|NULL|329.6|329.1|329.7|328.3|
|  3|NULL|332.8|334.1|334.3|332.0|
|  4|NULL|445.3|446.8|449.0|443.6|
|  5|NULL|449.2|448.5|450.5|448.0|
|  6|NULL|413.3|415.0|416.2|412.8|
|  7|NULL|396.3|399.7|400.8|394.6|
|  8|NULL|405.5|401.0|407.3|400.0|
|  9|NULL|336.1|333.5|336.5|333.0|
| 10|NULL|362.0|366.0|367.5|360.5|
+---+----+-----+-----+-----+-----+
only showing top 10 rows



In [5]:
df_gold_bronze = spark.read.format("iceberg").table("datalake.bronze.gold")
df_gold_bronze.show(10)

+----------+-----+-----+-----+-----+------+------+
|      Date|Price| Open| High|  Low|   Vol|Change|
+----------+-----+-----+-----+-----+------+------+
|12/30/1993|391.9|393.3|393.9|391.0|29.64K| 0.03%|
|12/29/1993|391.8|386.6|392.5|384.8|18.71K| 0.85%|
|12/28/1993|388.5|389.7|390.0|387.9|39.20K|-0.15%|
|12/27/1993|389.1|388.1|389.4|387.7| 8.38K| 0.54%|
|12/23/1993|387.0|387.0|387.3|385.6| 6.91K|-0.05%|
|12/22/1993|387.2|387.3|388.2|386.0| 8.37K|-0.54%|
|12/21/1993|389.3|390.4|390.5|388.5|15.93K|-0.38%|
|12/20/1993|390.8|391.1|392.2|390.6|13.58K| 0.49%|
|12/17/1993|388.9|387.3|389.1|386.8|16.35K| 0.36%|
|12/16/1993|387.5|388.3|389.6|386.5|19.51K|-0.39%|
+----------+-----+-----+-----+-----+------+------+
only showing top 10 rows



In [10]:
df_gold_bronze.schema["Date"].dataType

StringType()

In [13]:
df_gold_bronze = df_gold_bronze.withColumn("Date", to_timestamp(col("Date")))
df_gold_bronze.schema["Date"].dataType

TimestampType()

In [ ]:
print("Bronze table Date column type:", df_gold_bronze.schema["Date"].dataType)
print("Silver table Date column type:", df_gold_silver.schema["Date"].dataType)

In [16]:
df_gold_bronze.schema["Date"]

StructField('Date', TimestampType(), True)